#### **Vector Stores**  
A system designed to store and retrieve data represented as numerical vectors.

Used in applications involving semantic search, recommendation engines, RAG (Retrieval-Augmented Generation), and multimedia search.

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

In [8]:
doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [9]:
docs = [doc1, doc2, doc3, doc4, doc5]
print(docs)

[Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'), Document(metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."), Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'), Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'), Document(metadata={'team': 'Chennai

In [10]:
# Creating a vector Store

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db",
    collection_name="cricket_players"
)

In [ ]:
# Add Documents to the vector store
vector_store.add_documents(docs)  # Assigns a unique ID to each document

['e6b4b460-6b23-4b03-bb20-889c2db57ac9',
 '676452be-a13e-4abe-ae71-be85ccb7260c',
 '65157bd3-0239-48c3-b045-5900c39c5efe',
 'd947d697-9719-4f4f-96a5-67024fb74521',
 'bcb23264-ed99-4235-bbee-9e364b1c1bc4']

In [12]:
# View Documents
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['e6b4b460-6b23-4b03-bb20-889c2db57ac9',
  '676452be-a13e-4abe-ae71-be85ccb7260c',
  '65157bd3-0239-48c3-b045-5900c39c5efe',
  'd947d697-9719-4f4f-96a5-67024fb74521',
  'bcb23264-ed99-4235-bbee-9e364b1c1bc4'],
 'embeddings': array([[ 0.00994726,  0.06914335, -0.05147117, ..., -0.0354334 ,
          0.01284807,  0.0124829 ],
        [ 0.00127749,  0.03129849, -0.02375384, ..., -0.00518366,
         -0.03280614,  0.02737715],
        [-0.10265914,  0.02650809,  0.022715  , ..., -0.03359753,
         -0.07984939, -0.0150771 ],
        [ 0.02123392, -0.02468548, -0.04494369, ..., -0.10995808,
          0.00572556,  0.09915382],
        [ 0.01873978,  0.04382844, -0.04304257, ..., -0.07801622,
         -0.07840679, -0.00304192]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [13]:
# Search Documents
vector_store.similarity_search(
    query="Who among these players is an Bowler",
    k=2
)

[Document(id='d947d697-9719-4f4f-96a5-67024fb74521', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='bcb23264-ed99-4235-bbee-9e364b1c1bc4', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [14]:
# Search with Similarity Score

vector_store.similarity_search_with_score(
    query="Who among these players is an Bowler",
    k=2
)

[(Document(id='d947d697-9719-4f4f-96a5-67024fb74521', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  1.0220750570297241),
 (Document(id='bcb23264-ed99-4235-bbee-9e364b1c1bc4', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.094281554222107)]

In [16]:
# Metadata Filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Mumbai Indians"}
)

[(Document(id='676452be-a13e-4abe-ae71-be85ccb7260c', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.8651392459869385),
 (Document(id='d947d697-9719-4f4f-96a5-67024fb74521', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  1.9243816137313843)]

In [18]:
# Update Documents

updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='e6b4b460-6b23-4b03-bb20-889c2db57ac9', document=updated_doc1)
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['e6b4b460-6b23-4b03-bb20-889c2db57ac9',
  '676452be-a13e-4abe-ae71-be85ccb7260c',
  '65157bd3-0239-48c3-b045-5900c39c5efe',
  'd947d697-9719-4f4f-96a5-67024fb74521',
  'bcb23264-ed99-4235-bbee-9e364b1c1bc4'],
 'embeddings': array([[-0.0023375 ,  0.05902081, -0.04774045, ..., -0.07264051,
          0.0027678 , -0.00344089],
        [ 0.00127749,  0.03129849, -0.02375384, ..., -0.00518366,
         -0.03280614,  0.02737715],
        [-0.10265914,  0.02650809,  0.022715  , ..., -0.03359753,
         -0.07984939, -0.0150771 ],
        [ 0.02123392, -0.02468548, -0.04494369, ..., -0.10995808,
          0.00572556,  0.09915382],
        [ 0.01873978,  0.04382844, -0.04304257, ..., -0.07801622,
         -0.07840679, -0.00304192]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a sin

In [20]:
# Delete Documents
vector_store.delete(ids=['e6b4b460-6b23-4b03-bb20-889c2db57ac9'])

vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['676452be-a13e-4abe-ae71-be85ccb7260c',
  '65157bd3-0239-48c3-b045-5900c39c5efe',
  'd947d697-9719-4f4f-96a5-67024fb74521',
  'bcb23264-ed99-4235-bbee-9e364b1c1bc4'],
 'embeddings': array([[ 0.00127749,  0.03129849, -0.02375384, ..., -0.00518366,
         -0.03280614,  0.02737715],
        [-0.10265914,  0.02650809,  0.022715  , ..., -0.03359753,
         -0.07984939, -0.0150771 ],
        [ 0.02123392, -0.02468548, -0.04494369, ..., -0.10995808,
          0.00572556,  0.09915382],
        [ 0.01873978,  0.04382844, -0.04304257, ..., -0.07801622,
         -0.07840679, -0.00304192]]),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is considered one 